## Project Template (Deepnote)

You can use this notebook as a starting point for your class project and/or the extra exercise for graduate students in Deepnote. It comes set up with the dependencies we use in class, so you don't have to install these yourself.

**Notes:**
- To use this in your own workspace, click “Duplicate” for the project in Deepnote.
- Class dependencies, namely `drake` and the `manipulation` package, are already preinstalled in the Deepnote image for this course, as well as a bunch of other dependencies that these packages depend on or that we use in class
- To see the full list of installed dependencies, as well as version numbers etc, see this file: [pyproject.toml](https://github.com/RussTedrake/manipulation/blob/master/pyproject.toml).
  - **Note**: we install all the extra dependencies, including the `dev` dependencies, into the Docker image that this deepnote project runs on.


In [1]:
%load_ext autoreload
%autoreload 2
# ^ make it so we can change inner files


In [2]:

import numpy as np
from pydrake.all import (
    ContactModel,
    DiagramBuilder,
    MeshcatVisualizer,
    MultibodyPlant,
    Rgba,
    RigidTransform,
    RollPitchYaw,
    SceneGraph,
    Simulator,
    Sphere,
    StartMeshcat,
    VectorLogSink,
    VisualizationConfig,
)

from manipulation.station import (
    AddPointClouds,
    MakeHardwareStation,
    Parser,
)
from scripts.load_scenario import load_scenario

In [3]:
# Start meshcat for visualization
meshcat = StartMeshcat()
print("Click the link above to open Meshcat in your browser!")

INFO:drake:Meshcat listening for connections at http://localhost:7000


Click the link above to open Meshcat in your browser!


In [14]:
use_hydroelastic = False # this takes forever

scenario = load_scenario()

# since makehardwarestation finalizes the plant before giving it back to us
def callback(parser: Parser):
    plant = parser.plant()
    plant.set_contact_model(ContactModel.kHydroelasticWithFallback)
    scene_graph: SceneGraph = parser.scene_graph()
    config = scene_graph.get_config()
    if use_hydroelastic:
        config.default_proximity_properties.compliance_type = "compliant"
    scene_graph.set_config(config)

station = MakeHardwareStation(scenario, meshcat=meshcat, parser_prefinalize_callback=callback)
plant: MultibodyPlant = station.plant()


builder = DiagramBuilder()
builder.AddSystem(station)

to_point_cloud = AddPointClouds(
    scenario=scenario, station=station, builder=builder, meshcat=meshcat
)
builder.ExportOutput(
    to_point_cloud["camera0"].point_cloud_output_port(), "point_cloud0"
)

# diagram = builder.Build()


OutputPortIndex(0)

In [5]:
from scripts.grasp_broom import plan_path, build_temp_plant

# basic test
# X_WG1 = RigidTransform(RollPitchYaw(np.pi/2, 0, 0), [0, 0.5, 1])
# X_WG2 = RigidTransform(RollPitchYaw(np.pi/2, 0, np.pi/2), [0, 1, 1])

# meshcat.SetObject("start", Sphere(0.02), rgba=Rgba(0.1, 0.9, 0.1, 1))
# meshcat.SetTransform("start", X_WG1)
# meshcat.SetObject("goal", Sphere(0.02), rgba=Rgba(0.1, 0.9, 0.1, 1))
# meshcat.SetTransform("goal", X_WG2)


In [6]:
# test function to see the temp plant

# diagram, plant, gripper_frame = build_temp_plant(q0 = None, meshcat = meshcat)
# config = VisualizationConfig(lcm_bus='default', publish_period=0.015625, publish_illustration=True, default_illustration_color=Rgba(r=0.9, g=0.9, b=0.9, a=1.0), publish_proximity=True, default_proximity_color=Rgba(r=0.8, g=0.0, b=0.0, a=1.0), initial_proximity_alpha=0.5, publish_contacts=True, publish_inertia=True, enable_meshcat_creation=True, delete_on_initialization_event=True, enable_alpha_sliders=False)
# context = diagram.CreateDefaultContext()
# diagram.ForcedPublish(context)
# meshcat.SetObject("gripper", Sphere(0.02), rgba=Rgba(0.1, 0.9, 0.1, 1))
# meshcat.SetTransform("gripper", gripper_frame.GetPoseInWorld(context))


In [7]:
# path = plan_path(X_WG1, X_WG2)

In [ ]:
from pydrake.systems.primitives import TrajectorySource, ConstantVectorSource
from scripts.ik import solve_ik_for_pose

# grasp broom
X_WStart = RigidTransform(RollPitchYaw(np.pi/2, 0, 0), [0, 1.0, 0.5])
X_WGoal = RigidTransform(RollPitchYaw(np.pi/2, 0, 0), [0.6, 1.8, 0.5])

meshcat.SetObject("start", Sphere(0.02), rgba=Rgba(0.1, 0.9, 0.1, 1))
meshcat.SetTransform("start", X_WStart)
meshcat.SetObject("goal", Sphere(0.02), rgba=Rgba(0.5, 0.9, 0.1, 1))
meshcat.SetTransform("goal", X_WGoal)

print(solve_ik_for_pose(plant, X_WStart))
print(solve_ik_for_pose(plant, X_WGoal))

traj = plan_path(X_WStart, X_WGoal, meshcat)

# print 20 joint positions sampled
times = np.linspace(traj.start_time(), traj.end_time(), 20)
for t in times:
    print(f'time{t}: {traj.value(t).flatten()}')

# add traj to builder and connect
q_src = builder.AddSystem(TrajectorySource(traj))
builder.Connect(q_src.get_output_port(), station.GetInputPort("iiwa.position"))
wsg_src = builder.AddSystem(ConstantVectorSource([1.0]))
builder.Connect(wsg_src.get_output_port(), station.GetInputPort("wsg.position"))

diagram = builder.Build()


(np.float64(0.7509546369713288), np.float64(-1.2387290426721662), np.float64(-1.9173577764712293), np.float64(-1.7818097620813607), np.float64(-1.2885017624919541), np.float64(-1.9731773752368413), np.float64(-0.8165490476037579))
(np.float64(0.24921523438529805), np.float64(2.09439510239), np.float64(-0.4382612331947843), np.float64(1.1164400811315451), np.float64(0.4136581813805621), np.float64(-1.0868616445544523), np.float64(-2.248966081230783))
7
time0.0: [-0.9188558  -0.11781019  0.98464777 -2.0356923   1.90958995 -0.23320782
  0.        ]
time0.025659360922810515: [-0.90467212 -0.13158913  0.99912092 -2.02715008  1.9096133  -0.22604623
  0.        ]
time0.05131872184562103: [-0.87006394 -0.16520975  1.03443538 -2.00630707  1.90967028 -0.20857194
  0.        ]
time0.07697808276843154: [-0.82689925 -0.20714271  1.07848099 -1.98031084  1.90974134 -0.18677734
  0.        ]
time0.10263744369124206: [-0.7830109  -0.24977865  1.12326502 -1.95387877  1.90981359 -0.16461736
  0.        ]

RuntimeError: DiagramBuilder: Build() or BuildInto() has already been called to create a Diagram; this DiagramBuilder may no longer be used.

In [24]:

simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0)
meshcat.StartRecording()
simulator.Initialize()
simulator.AdvanceTo(traj.end_time())
meshcat.StopRecording()
meshcat.PublishRecording()

KeyboardInterrupt: 